Test Cases covered in this .ipynb file

1. SVM Pretrained Binary (model1=pretrain)
2. SVM Custom Binary (model2=custom)
3. Perceptron Pretrained Binary
4. Perceptron Custom Binary
5. SVM TF/IDF Binary
6. Perceptron TF/IDF Binary
7. (4a)FFNN AVG Custom Binary
8. (4a)FFNN AVG Custom Ternary
9. (4b)FFNN CONCAT Custom Binary

Remaining will be covered in different .ipynb due to memory constraints

## Import the required packages

In [1]:
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/payalrashinkar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/payalrashinkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Get the dataset into dataframe

In [2]:
df_1 = pd.read_csv('https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', on_bad_lines='skip', low_memory=False)

# <span style="color:red">1. Data Generation</span>

### 1.a Copy the df_1 into df to get only required 2 columns i.e 'ratings' and 'reviews' after renaming them:

In [3]:
df = df_1.copy()
df = df[['star_rating', 'review_body']].rename(columns={'star_rating':'ratings', 'review_body':'reviews'})
df.head()

ratings                                            reviews
0       5                                     Great product.
1       5  What's to say about this commodity item except...
2       5    Haven't used yet, but I am sure I will like it.
3       1  Although this was labeled as &#34;new&#34; the...
4       4                    Gorgeous colors and easy to use

### <span style="color:blue">1.a Solution:</span> 
<span style="color:green">
    Copying the dataframe df_1 to df and only have two columns in df i.e. ratings and reviews after renaming it from start_rating and review_body respectively.
</span>

### 1.b Creating 50k dataset randomly for each of 5 ratings:

In [4]:
balanced_df = pd.DataFrame()
for ratings in ['1','2', '3', '4', '5']:
    sampled_df = df[df['ratings'] == ratings].sample(n=50000, random_state=42)
    balanced_df = pd.concat([balanced_df, sampled_df])
    print(f"Rating {ratings}: {len(sampled_df)} datasets")

Rating 1: 50000 datasets
Rating 2: 50000 datasets
Rating 3: 50000 datasets
Rating 4: 50000 datasets
Rating 5: 50000 datasets


### <span style="color:blue">1.b Solution:</span> 
<span style="color:green">
    1. The code iterates over a list of ratings ('1', '2', '3', '4', ‘5’). For each rating, it filters the DataFrame df to include only rows with the current rating, then samples 50,000 rows from this filtered subset using the sampling method with a fixed random_state for reproducibility.
    2. These sampled rows are concatenated to a new DataFrame balanced_df, creating a balanced dataset with an equal number of rows for each rating.
    3. After each iteration, the code prints the number of rows sampled for the current rating, summarizing how many datasets are included for each rating in balanced_df.
</span>

### 1.c Creating column called "class" to group the sentiment based on rating:

In [5]:
# Create ternary labels
def label_sentiment(row):
    if row['ratings'] > '3':
        return 1  # Positive
    elif row['ratings'] < '3':
        return 2  # Negative
    else:
        return 3  # Neutral
    

balanced_df['class'] = balanced_df.apply(label_sentiment, axis=1)

In [6]:
print(balanced_df.head())

        ratings                                            reviews  class
1846471       1  Staggering amount of &#34;negative reviews&#34...      2
957331        1  Worst labels ever! I purchased these labels to...      2
2474769       1  This product had to be returned. On the first ...      2
1045185       1  Two of the four ink cartridges were empty. You...      2
687232        1                      didn't even stay on one night      2


### <span style="color:blue">1.c Solution:</span> 
<span style="color:green">
    1. The code defines a function label_sentiment that assigns ternary labels (1 for Positive, 2 for Negative, 3 for Neutral) based on the 'ratings' column of each row in the balanced_df DataFrame, considering ratings higher than '3' as positive, lower than '3' as negative, and equal to '3' as neutral.
    2. It then applies this function to each row of balanced_df to create a new column 'class' with the corresponding sentiment labels and prints the first five rows of the DataFrame to display the output.
</span>

### 1.d Split the dataset into training and testing sets:

In [7]:
train_df, test_df = train_test_split(balanced_df, test_size=0.2, random_state=42)

In [8]:
print(f"80% Train data: {train_df['ratings'].count()} datasets")
print(f"20% Test data: {test_df['ratings'].count()} datasets")

80% Train data: 200000 datasets
20% Test data: 50000 datasets


### <span style="color:blue">1.d Solution:</span> <span style="color:green">The code splits the balanced_df DataFrame into training and testing sets with an 80%/20% ratio using a fixed random_state for reproducibility and then prints the number of datasets (rows) in each resulting subset.</span>

# <span style="color:red">2. Word Embedding</span>

In [9]:
# load the pretrained model as model1
pretrained = 'word2vec-google-news-300.gz'
model1 = KeyedVectors.load_word2vec_format(pretrained, binary=True)


In [10]:
# check the similarity between two similar words
print(model1.similarity('excellent', 'outstanding'))

# find out the corresponding word given that A - B = C - D (King - Man = Queen - Woman)
print(model1.most_similar(positive=['king', 'woman'], negative=['man'], topn=1))


0.55674857
[('queen', 0.7118193507194519)]


In [11]:
X, y = balanced_df['reviews'].fillna('').tolist(), balanced_df['class'].tolist()

In [12]:
# convert reviews to lower case
X = [str(x).lower() for x in X]
# remove HTML and URLs from reviews
X = [re.sub('<.*>', '', x) for x in X]
X = [re.sub(r'https?://\S+', '', x) for x in X]
# remove non-alphabetical characters
X = [re.sub('[^a-z ]', '', x) for x in X]
# remove extra spaces
X = [re.sub(' +', ' ', x) for x in X]

In [13]:
# expand contractions
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
def decontraction(s):
    for word in s.split(' '):
        if word in contractions.keys():
            s = re.sub(word, contractions[word], s)
    return s
X = [decontraction(x) for x in X]

In [14]:
# remove stop words
stopWords =set(stopwords.words('english'))
def remvstopWords(s):
    wordlist = s.split(' ')
    newlist = []
    for word in wordlist:
        if word not in stopWords:
            newlist.append(word)
    s = ' '.join(newlist)
    return s

X = list(map(remvstopWords, X)) 

In [15]:
# perform lemmatization
wnl = WordNetLemmatizer()
X = [' '.join([wnl.lemmatize(word) for word in x.split(' ')]) for x in X]

In [16]:
# train a word2vec model using my own dataset
# convert X_train to a list of lists of words
sentences = [x.split(' ') for x in X]

# use X_train to train a word2vec model2
model2 = Word2Vec(vector_size=300, window=11, min_count=10)
model2.build_vocab(sentences)
model2.train(sentences, total_examples=model2.corpus_count, epochs=model2.epochs)


(29065883, 32427760)

In [17]:
# save the trained model
model2.save('my-own-word2vec.model')
# store just the words + their trained embeddings
word_vectors = model2.wv
word_vectors.save('my-own-word2vec.wordvectors')

In [18]:
model2 = KeyedVectors.load('my-own-word2vec.wordvectors', mmap='r')

In [19]:
# check the similarity between two similar words using model 2
print(model2.similarity('excellent', 'outstanding'))

# find out the corresponding word given that A - B = C - D (King - Man = Queen - Woman) using model 2
print(model2.most_similar(positive=['king', 'woman'], negative=['man'], topn=1))

0.79444706
[('graduated', 0.5376219153404236)]


### <span style="color:blue">2 Answer:</span> <span style="color:green">The custom model demonstrates superior performance over the pretrained model, especially after extensive data preprocessing steps such as cleaning, lemmatization, and expanding abbreviations. This indicates that our tailored approach outshines the generic pretrained model in terms of accuracy. Furthermore, it effectively measures the similarity between two closely related words, as illustrated previously, highlighting its nuanced understanding of word relationships.</span>

# <span style="color:red">3. Simple Models</span>

In [20]:
# Split the downsized dataset into 80% training dataset and 20% testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [21]:
# turn the original ternary training and testing datasets to binary
# return a list of indices of y = 1 or y =2
y_train_bi = []
idx_train = []
for i, y in enumerate(y_train):
    if y == 1 or y == 2:
        y_train_bi.append(y)
        idx_train.append(i)
        
y_test_bi = []
idx_test = []
for i, y in enumerate(y_test):
    if y == 1 or y == 2:
        y_test_bi.append(y)
        idx_test.append(i)
# use the list of indices to select a sub-list of X_train
X_train_bi = [X_train[i] for i in idx_train]
X_test_bi = [X_test[i] for i in idx_test]

In [22]:
# take the average word vectors of important words (i.e. non-stop words) in
# a review as the feature of a training sample
X_train_bi1 = []
for x in X_train_bi:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = model1[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_train_bi1.append(np.mean(wordveclist, axis=0)) 

/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [23]:
X_test_bi1 = []
for x in X_test_bi:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = model1[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_test_bi1.append(np.mean(wordveclist, axis=0)) 

In [24]:
# handle non-word-vector values in the dataset
# return the indices of word-vector values
wv_train = []
for i, x in enumerate(X_train_bi1):
    try:
        len(x)
        wv_train.append(i)
    except:
        pass
        

wv_test = []
for i, x in enumerate(X_test_bi1):
    try:
        len(x)
        wv_test.append(i)
    except:
        pass
        
# remove the non-word-vector values from the dataset
X_train_bi1 = [X_train_bi1[i] for i in wv_train]
X_test_bi1 = [X_test_bi1[i] for i in wv_test]
y_train_bi1 = [y_train_bi[i] for i in wv_train]
y_test_bi1 = [y_test_bi[i] for i in wv_test]

### 3.1a Pretrained word2vec google model for perceptron:

In [25]:
# use pretrained word2vec features to train a perceptron
perceptron = Perceptron(random_state=1)
perceptron.fit(X_train_bi1, y_train_bi1)
y_train_predict1, y_test_predict1 = perceptron.predict(X_train_bi1), perceptron.predict(X_test_bi1)

# report accuracy, precision, recall, and f1-score on both the training and testing split
train_stats = precision_recall_fscore_support(y_train_bi1, y_train_predict1, average='binary')
precision_train, recall_train, fscore_train = train_stats[0], train_stats[1], train_stats[2]

test_stats = precision_recall_fscore_support(y_test_bi1, y_test_predict1, average='binary')
precision_test, recall_test, fscore_test = test_stats[0], test_stats[1], test_stats[2]

print('The accuracy of testing dataset: {:2.1%}'.format(perceptron.score(X_test_bi1, y_test_bi1)))

The accuracy of testing dataset: 70.1%


### 3.1b Pretrained word2vec google model for SVM:

In [26]:
svm = LinearSVC(random_state=1)
svm.fit(X_train_bi1, y_train_bi1)

y_train_predict1, y_test_predict1 = svm.predict(X_train_bi1), svm.predict(X_test_bi1)

# report accuracy, precision, recall, and f1-score on both the training and testing split
train_stats = precision_recall_fscore_support(y_train_bi1, y_train_predict1, average='binary')
precision_train, recall_train, fscore_train = train_stats[0], train_stats[1], train_stats[2]

test_stats = precision_recall_fscore_support(y_test_bi1, y_test_predict1, average='binary')
precision_test, recall_test, fscore_test = test_stats[0], test_stats[1], test_stats[2]

print('The accuracy of testing dataset: {:2.1%}'.format(svm.score(X_test_bi1, y_test_bi1)))

/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


The accuracy of testing dataset: 81.1%


In [27]:
# take the average word vectors of important words (i.e. non-stop words) in
# a review as the feature of a training sample
X_train_bi2 = []
for x in X_train_bi:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = model2[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_train_bi2.append(np.mean(wordveclist, axis=0))

# do the same to the testing dataset
X_test_bi2 = []
for x in X_test_bi:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = model2[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_test_bi2.append(np.mean(wordveclist, axis=0)) 

/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [58]:
# handle non-word-vector values in the dataset
# return the indices of word-vector values
wv_train = []
for i, x in enumerate(X_train_bi2):
    try:
        len(x)
        wv_train.append(i)
    except:
        pass
        

wv_test = []
for i, x in enumerate(X_test_bi2):
    try:
        len(x)
        wv_test.append(i)
    except:
        pass
        
# remove the non-word-vector values from the dataset
X_train_bi2 = [X_train_bi2[i] for i in wv_train]
X_test_bi2 = [X_test_bi2[i] for i in wv_test]
y_train_bi2 = [y_train_bi[i] for i in wv_train]
y_test_bi2 = [y_test_bi[i] for i in wv_test]

### 3.2a Our word2vec model for perceptron:

In [29]:
# use my own word2vec features to train a perceptron
perceptron = Perceptron(random_state=1)
perceptron.fit(X_train_bi2, y_train_bi2)
y_train_predict2, y_test_predict2 = perceptron.predict(X_train_bi2), perceptron.predict(X_test_bi2)

# report accuracy, precision, recall, and f1-score on both the training and testing split
train_stats = precision_recall_fscore_support(y_train_bi2, y_train_predict2, average='binary')
precision_train, recall_train, fscore_train = train_stats[0], train_stats[1], train_stats[2]

test_stats = precision_recall_fscore_support(y_test_bi2, y_test_predict2, average='binary')
precision_test, recall_test, fscore_test = test_stats[0], test_stats[1], test_stats[2]

print('The accuracy of testing dataset: {:2.1%}'.format(perceptron.score(X_test_bi2, y_test_bi2)))

The accuracy of testing dataset: 70.9%


### 3.2b Our word2vec model for SVM:

In [30]:
svm = LinearSVC(random_state=1)
svm.fit(X_train_bi2, y_train_bi2)

y_train_predict2, y_test_predict2 = svm.predict(X_train_bi2), svm.predict(X_test_bi2)

# report accuracy, precision, recall, and f1-score on both the training and testing split
train_stats = precision_recall_fscore_support(y_train_bi2, y_train_predict2, average='binary')
precision_train, recall_train, fscore_train = train_stats[0], train_stats[1], train_stats[2]

test_stats = precision_recall_fscore_support(y_test_bi2, y_test_predict2, average='binary')
precision_test, recall_test, fscore_test = test_stats[0], test_stats[1], test_stats[2]

print('The accuracy of testing dataset: {:2.1%}'.format(svm.score(X_test_bi2, y_test_bi2)))

/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


The accuracy of testing dataset: 84.1%


### 3.3a Our TF-IDF model for perceptron and SVM from homework1:

In [31]:
# As executed from Homework1:
#============================
     
#Perceptron for TF-IDF
#=====================
#Perceptron, accuracy of testing dataset: 84.4%
   
#SVM for TF-IDF
#===============
#SVM, accuracy of testing dataset: 88.7%

### <span style="color:blue">3 Answer:</span> <span style="color:green"> In the context of the specified classification task, features derived from TF-IDF demonstrate greater efficacy for the Perceptron and SVM models than those obtained from Word2Vec. The superior accuracy rates associated with TF-IDF indicate its enhanced capability in encapsulating the essential information required for classification within this specific scenario. Nonetheless, it's crucial to recognize that the selection of feature type might vary based on the unique characteristics of the data and the task at hand, implying that various tasks might favor distinct types of feature representations.</span>

# <span style="color:red">4. Feedforward Neural Network</span>

In [32]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import functools
from sklearn.metrics import accuracy_score 

In [33]:
# Enable CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True


In [34]:
# Hyperparameters
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 0}
max_epochs = 10

In [35]:
# Override the Dataset class
class Dataset(Dataset):
    
    def __init__(self, list_IDs, labels):
        'Initialization'
        self.list_IDs = list_IDs
        self.labels = labels
    
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]
        
        # Load data and get label
        X = torch.load('data/' + ID + '.pt')
        y = self.labels[index]
        return X, y

Use the average of the Word2Vec vectors for each review as the input feature


### <span style="color:blue">4a</span> <span style="color:green"></span>
### <span style="color:blue">1. Binary classification custom model:</span> <span style="color:green"></span>

In [36]:
train_IDs = {}
y_train = []
len_train_IDs = int(0.8 * len(X_train_bi2))
#print(len_train_IDs)
for i in range(len_train_IDs):
    train_IDs[i] = 'train_bi_' + str(i)
    y_train.append(y_train_bi2[i] - 1) # Convert label from 1 and 2 to 0 and 1

#print(y_train)
valid_IDs = {}
y_valid = []
len_valid_IDs = len(X_train_bi2) - len_train_IDs
for i in range(len_valid_IDs):
    valid_IDs[i] = 'valid_bi_' + str(i)
    y_valid.append(y_train_bi2[len(y_train) + i] - 1)
    
test_IDs = {}
len_test_IDs = len(X_test_bi2)
for i in range(len_test_IDs):
    test_IDs[i] = 'test_bi_' + str(i)

for i in range(len(train_IDs)):
    torch.save(X_train_bi2[i], 'data/' + train_IDs[i] + '.pt')

for i in range(len(valid_IDs)):
    torch.save(X_train_bi2[len(train_IDs) + i], 'data/' + valid_IDs[i] + '.pt')

for i in range(len(test_IDs)):
    torch.save(X_test_bi2[i], 'data/' + test_IDs[i] + '.pt')    

In [37]:
train_set = Dataset(train_IDs, y_train)
valid_set = Dataset(valid_IDs, y_valid)
test_set = Dataset(test_IDs, y_test_bi2)

# Generate dataloaders for the training, validation and testing datasets
train_loader = torch.utils.data.DataLoader(train_set, **params)
valid_loader = torch.utils.data.DataLoader(valid_set, **params)
test_loader = torch.utils.data.DataLoader(test_set, **params)

In [38]:
# Define the network architecture
class Net(nn.Module):
    def __init__(self, input_dim=300, output_dim=2, hidden_1=50, hidden_2=10):
        super(Net, self).__init__()
        # dimension of inputs
        self.input_dim = input_dim 
        # number of classes (2 for binary, 3 for ternary, ...)
        self.output_dim = output_dim
        # number of nodes in each hidden layer
        self.hidden_1 = hidden_1
        self.hidden_2 = hidden_2
        # linear layer (input --> hidden_1)
        self.fc1 = nn.Linear(self.input_dim, self.hidden_1)
        # linear layer (hidden_1 --> hidden_2)
        self.fc2 = nn.Linear(self.hidden_1, self.hidden_2)
        # linear layer (hidden_2 --> 2)
        self.fc3 = nn.Linear(self.hidden_2, self.output_dim)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        # add the 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer after the 1st hidden layer
        x = self.dropout(x)
        # add the 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer after the 2nd hidden layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x  

In [39]:
# initialize the NN
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [40]:
# specify cross entropy loss as loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [41]:
# train the network

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(max_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train() # prep model for training
    for data, target in train_loader:
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    model.eval() # prep model for evaluation
    for data, target in valid_loader:
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.541041 	Validation Loss: 0.412022
Validation loss decreased (inf --> 0.412022).  Saving model ...
Epoch: 2 	Training Loss: 0.417179 	Validation Loss: 0.383492
Validation loss decreased (0.412022 --> 0.383492).  Saving model ...
Epoch: 3 	Training Loss: 0.397067 	Validation Loss: 0.374651
Validation loss decreased (0.383492 --> 0.374651).  Saving model ...
Epoch: 4 	Training Loss: 0.390049 	Validation Loss: 0.369608
Validation loss decreased (0.374651 --> 0.369608).  Saving model ...
Epoch: 5 	Training Loss: 0.383501 	Validation Loss: 0.365544
Validation loss decreased (0.369608 --> 0.365544).  Saving model ...
Epoch: 6 	Training Loss: 0.379078 	Validation Loss: 0.361988
Validation loss decreased (0.365544 --> 0.361988).  Saving model ...
Epoch: 7 	Training Loss: 0.376475 	Validation Loss: 0.360423
Validation loss decreased (0.361988 --> 0.360423).  Saving model ...
Epoch: 8 	Training Loss: 0.372543 	Validation Loss: 0.358516
Validation loss decreased (0.36042

In [42]:
## Test the trained network
def predict(model, dataloader):
    prediction_list = []
    truth_list = []
    for i, batch in enumerate(dataloader):
        # batch[0] is X, and batch[1] is y
        outputs = model(batch[0])
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted)
        truth_list.append(batch[1])
    return prediction_list, truth_list

In [43]:
predictions, truths = predict(model, test_loader)
# Convert predictions and truths from list of tensors to list of lists
predictions = [list(torch.Tensor.numpy(t)) for t in predictions]
truths = [list(torch.Tensor.numpy(t)) for t in truths]
# Convert predictions and truths from list of lists to a single list
predictions = functools.reduce(lambda a, b: a + b, predictions)
truths = functools.reduce(lambda a, b: a + b, truths)
# Convert predictions from (0, 1) to (1, 2)
predictions = [p + 1 for p in predictions]

In [44]:
# report accuracy, precision, recall, and f1-score on the testing dataset

test_stats = precision_recall_fscore_support(truths, predictions, average='binary')
precision_test, recall_test, fscore_test = test_stats[0], test_stats[1], test_stats[2]

print('The accuracy of testing dataset: {:2.1%}'.format(accuracy_score(truths, predictions)))

The accuracy of testing dataset: 85.0%


### <span style="color:blue">2 Ternary Classification custom model</span> <span style="color:green"></span>

In [45]:
X, y = balanced_df['reviews'].fillna('').tolist(), balanced_df['class'].tolist()

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [47]:
X_train_te2 = []
for x in X_train:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = model2[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_train_te2.append(np.mean(wordveclist, axis=0))

# do the same to the testing dataset
X_test_te2 = []
for x in X_test:
    wordveclist = []
    for word in x.split(' '):
        try:
            wordvec = model2[word]
            wordveclist.append(wordvec)
        except:
            pass
    X_test_te2.append(np.mean(wordveclist, axis=0)) 

/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/payalrashinkar/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [48]:
# handle non-word-vector values in the dataset
# return the indices of word-vector values
wv_train = []
for i, x in enumerate(X_train_te2):
    try:
        len(x)
        wv_train.append(i)
    except:
        pass
        

wv_test = []
for i, x in enumerate(X_test_te2):
    try:
        len(x)
        wv_test.append(i)
    except:
        pass
        
# remove the non-word-vector values from the dataset
X_train_te2 = [X_train_te2[i] for i in wv_train]
X_test_te2 = [X_test_te2[i] for i in wv_test]
y_train_te2 = [y_train[i] for i in wv_train]
y_test_te2 = [y_test[i] for i in wv_test]

In [49]:
train_IDs = {}
y_train = []
len_train_IDs = int(0.8 * len(X_train_te2))

for i in range(len_train_IDs):
    train_IDs[i] = 'train_te_' + str(i)
    y_train.append(y_train_te2[i] - 1) # Convert label from 1, 2 and 3 to 0, 1 and 2



valid_IDs = {}
y_valid = []
len_valid_IDs = len(X_train_te2) - len_train_IDs
for i in range(len_valid_IDs):
    valid_IDs[i] = 'valid_te_' + str(i)
    y_valid.append(y_train_te2[len(y_train) + i] - 1)
    
test_IDs = {}
len_test_IDs = len(X_test_te2)
for i in range(len_test_IDs):
    test_IDs[i] = 'test_te_' + str(i)

for i in range(len(train_IDs)):
    torch.save(X_train_te2[i], 'data/' + train_IDs[i] + '.pt')

for i in range(len(valid_IDs)):
    torch.save(X_train_te2[len(train_IDs) + i], 'data/' + valid_IDs[i] + '.pt')

for i in range(len(test_IDs)):
    torch.save(X_test_te2[i], 'data/' + test_IDs[i] + '.pt')

In [50]:
# Generate training, validation and testing datasets
train_set = Dataset(train_IDs, y_train)
valid_set = Dataset(valid_IDs, y_valid)
test_set = Dataset(test_IDs, y_test_te2)

# Generate dataloaders for the training, validation and testing datasets
train_loader = torch.utils.data.DataLoader(train_set, **params)
valid_loader = torch.utils.data.DataLoader(valid_set, **params)
test_loader = torch.utils.data.DataLoader(test_set, **params)

In [51]:
# initialize the NN
# use model_te to be distinguishable from the binary model
model_te = Net(output_dim=3)
print(model_te)

Net(
  (fc1): Linear(in_features=300, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [52]:
# specify cross entropy loss as loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model_te.parameters(), lr=0.01)

In [53]:
# train the network

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(max_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    

    model_te.train() # prep model for training
    for data, target in train_loader:
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_te(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    model_te.eval() # prep model for evaluation
    for data, target in valid_loader:
        output = model_te(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model_te.state_dict(), 'model_te.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.017167 	Validation Loss: 0.940452
Validation loss decreased (inf --> 0.940452).  Saving model ...
Epoch: 2 	Training Loss: 0.912628 	Validation Loss: 0.873352
Validation loss decreased (0.940452 --> 0.873352).  Saving model ...
Epoch: 3 	Training Loss: 0.879003 	Validation Loss: 0.851746
Validation loss decreased (0.873352 --> 0.851746).  Saving model ...
Epoch: 4 	Training Loss: 0.866532 	Validation Loss: 0.841702
Validation loss decreased (0.851746 --> 0.841702).  Saving model ...
Epoch: 5 	Training Loss: 0.858859 	Validation Loss: 0.836471
Validation loss decreased (0.841702 --> 0.836471).  Saving model ...
Epoch: 6 	Training Loss: 0.853147 	Validation Loss: 0.831773
Validation loss decreased (0.836471 --> 0.831773).  Saving model ...
Epoch: 7 	Training Loss: 0.849154 	Validation Loss: 0.827806
Validation loss decreased (0.831773 --> 0.827806).  Saving model ...
Epoch: 8 	Training Loss: 0.844932 	Validation Loss: 0.824553
Validation loss decreased (0.82780

In [54]:
# Test the trained network
# Use the same predict function as the binary case
def predict(model, dataloader):
    prediction_list = []
    truth_list = []
    for i, batch in enumerate(dataloader):
        # batch[0] is X, and batch[1] is y
        outputs = model(batch[0])
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted)
        truth_list.append(batch[1])
    return prediction_list, truth_list

In [56]:
# Load model parameters from the trained model with the lowest validation loss
model_te.load_state_dict(torch.load('model_te.pt'))
predictions, truths = predict(model_te, test_loader)
# Convert predictions and truths from list of tensors to list of lists
predictions = [list(torch.Tensor.numpy(t)) for t in predictions]
truths = [list(torch.Tensor.numpy(t)) for t in truths]
# Convert predictions and truths from list of lists to a single list
predictions = functools.reduce(lambda a, b: a + b, predictions)
truths = functools.reduce(lambda a, b: a + b, truths)
# Convert predictions from (0, 1, 2) to (1, 2, 3)
predictions = [p + 1 for p in predictions]

In [57]:
# report accuracy, precision, recall, and f1-score on the testing dataset

test_stats = precision_recall_fscore_support(truths, predictions, average='micro')
precision_test, recall_test, fscore_test = test_stats[0], test_stats[1], test_stats[2]

print('The accuracy of testing dataset: {:2.1%}'.format(accuracy_score(truths, predictions)))

The accuracy of testing dataset: 64.5%


# 4.b Use the concatenation of the first 10 Word2Vec vectors for each review as the input feature


### <span style="color:blue">1 Binary Classification custom model</span> <span style="color:green"></span>

In [59]:
%%time
X_train_bi2 = []
for x in X_train_bi:
    wordveclist = []
    first_10_words = x.split(' ')[:10]
    for word in first_10_words:
        try:
            wordvec = list(model2[word])
            wordveclist.append(wordvec)
        except:
            pass
            
    X_train_bi2.append(wordveclist)

# do the same to the testing dataset
X_test_bi2 = []
for x in X_test_bi:
    wordveclist = []
    first_10_words = x.split(' ')[:10]
    for word in first_10_words:
        try:
            wordvec = list(model2[word])
            wordveclist.append(wordvec)
        except:
            pass
            
    X_test_bi2.append(wordveclist) 

CPU times: user 1min 3s, sys: 2min 13s, total: 3min 16s
Wall time: 6min 29s


In [60]:
# define a function that adds padding to the reviews with length less than 10 words
def add_padding(x):
    padding = [0 for _ in range(300)]
    if len(x) < 10:
        for i in range(10 - len(x)):
            x.append(padding)
    return x

# add paddings
X_train_bi2 = [add_padding(x) for x in X_train_bi2]
X_test_bi2 = [add_padding(x) for x in X_test_bi2]

In [61]:
# reshape the word vector of a review from (10, 300) to (3000,)
X_train_bi2 = [functools.reduce(lambda a, b: a + b, x) for x in X_train_bi2]
X_test_bi2 = [functools.reduce(lambda a, b: a + b, x) for x in X_test_bi2]

In [62]:
len_train = int(0.8 * len(X_train_bi2))
X_train, y_train = X_train_bi2[:len_train], []
for i in range(len_train):
    y_train.append(y_train_bi[i] - 1) # Convert label from 1 and 2 to 0 and 1

len_valid = len(X_train_bi2) - len_train
X_valid, y_valid = X_train_bi2[len_train:], []
for i in range(len_valid):
    y_valid.append(y_train_bi[len_train + i] - 1)

X_test, y_test = X_test_bi2, y_test_bi

In [63]:
# Override the Dataset class
class Dataset(Dataset):
    
    def __init__(self, features, labels):
        'Initialization'
        self.features = features
        self.labels = labels
    
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.features)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = torch.tensor(self.features[index])
        y = self.labels[index]
        
        return X, y  

In [64]:
# Generate training, validation and testing datasets
train_set = Dataset(X_train, y_train)
valid_set = Dataset(X_valid, y_valid)
test_set = Dataset(X_test, y_test)

# Generate dataloaders for the training, validation and testing datasets
train_loader = torch.utils.data.DataLoader(train_set, **params)
valid_loader = torch.utils.data.DataLoader(valid_set, **params)
test_loader = torch.utils.data.DataLoader(test_set, **params)

In [65]:
# Define the network architecture
# The same as before
class Net(nn.Module):
    def __init__(self, input_dim=300, output_dim=2, hidden_1=50, hidden_2=10):
        super(Net, self).__init__()
        # dimension of inputs
        self.input_dim = input_dim 
        # number of classes (2 for binary, 3 for ternary, ...)
        self.output_dim = output_dim
        # number of nodes in each hidden layer
        self.hidden_1 = hidden_1
        self.hidden_2 = hidden_2
        # linear layer (input --> hidden_1)
        self.fc1 = nn.Linear(self.input_dim, self.hidden_1)
        # linear layer (hidden_1 --> hidden_2)
        self.fc2 = nn.Linear(self.hidden_1, self.hidden_2)
        # linear layer (hidden_2 --> 2)
        self.fc3 = nn.Linear(self.hidden_2, self.output_dim)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        # add the 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer after the 1st hidden layer
        x = self.dropout(x)
        # add the 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        # add dropout layer after the 2nd hidden layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x

In [66]:
# initialize the NN
model_bi2 = Net(input_dim=3000)
print(model_bi2)

Net(
  (fc1): Linear(in_features=3000, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [67]:
# specify cross entropy loss as loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model_bi2.parameters(), lr=0.01)

In [68]:
# train the network

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(max_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0

    model_bi2.train() # prep model for training
    for data, target in train_loader:
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_bi2(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)

    model_bi2.eval() # prep model for evaluation
    for data, target in valid_loader:
        # transfer to GPU
        #data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_bi2(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model_bi2.state_dict(), 'model_bi2.pt')
        valid_loss_min = valid_loss       

Epoch: 1 	Training Loss: 0.529863 	Validation Loss: 0.467470
Validation loss decreased (inf --> 0.467470).  Saving model ...
Epoch: 2 	Training Loss: 0.460842 	Validation Loss: 0.457045
Validation loss decreased (0.467470 --> 0.457045).  Saving model ...
Epoch: 3 	Training Loss: 0.442386 	Validation Loss: 0.451827
Validation loss decreased (0.457045 --> 0.451827).  Saving model ...
Epoch: 4 	Training Loss: 0.427531 	Validation Loss: 0.451010
Validation loss decreased (0.451827 --> 0.451010).  Saving model ...
Epoch: 5 	Training Loss: 0.415802 	Validation Loss: 0.448439
Validation loss decreased (0.451010 --> 0.448439).  Saving model ...
Epoch: 6 	Training Loss: 0.403751 	Validation Loss: 0.450240
Epoch: 7 	Training Loss: 0.393425 	Validation Loss: 0.450338
Epoch: 8 	Training Loss: 0.381452 	Validation Loss: 0.454666
Epoch: 9 	Training Loss: 0.371288 	Validation Loss: 0.458266
Epoch: 10 	Training Loss: 0.362224 	Validation Loss: 0.464123


In [69]:
# Test the trained network
# The same as before
def predict(model, dataloader):
    prediction_list = []
    truth_list = []
    for i, batch in enumerate(dataloader):
        # batch[0] is X, and batch[1] is y
        outputs = model(batch[0])
        _, predicted = torch.max(outputs.data, 1)
        prediction_list.append(predicted)
        truth_list.append(batch[1])
    return prediction_list, truth_list


In [70]:
# Load model parameters from the trained model with the lowest validation loss
model_bi2.load_state_dict(torch.load('model_bi2.pt'))

predictions, truths = predict(model_bi2, test_loader)
# Convert predictions and truths from list of tensors to list of lists
predictions = [list(torch.Tensor.numpy(t)) for t in predictions]
truths = [list(torch.Tensor.numpy(t)) for t in truths]
# Convert predictions and truths from list of lists to a single list
predictions = functools.reduce(lambda a, b: a + b, predictions)
truths = functools.reduce(lambda a, b: a + b, truths)
# Convert predictions from (0, 1) to (1, 2)
predictions = [p + 1 for p in predictions]


In [71]:
# report accuracy, precision, recall, and f1-score on the testing dataset

test_stats = precision_recall_fscore_support(truths, predictions, average='binary')
precision_test, recall_test, fscore_test = test_stats[0], test_stats[1], test_stats[2]

print('The accuracy of testing dataset: {:2.1%}'.format(accuracy_score(truths, predictions)))

The accuracy of testing dataset: 79.5%


### <span style="color:blue">4 Answer:</span> <span style="color:green">The comparison of accuracy rates across various models shows that the SVM model, when trained using TF-IDF features, leads the pack with an impressive accuracy rate of 88.7%, showcasing its outstanding performance for this specific task. Following this, the Perceptron model that employs Word2Vec features records a respectable accuracy of 84.4%, indicating a solid performance. In the Feed Forward Neural Network (FNN) models, the variant that utilizes average Word2Vec vectors achieves a comparable accuracy of 80%, matching the SVM's performance but not exceeding that of the Perceptron model equipped with TF-IDF features. The FNN model is designed to concatenate the initial 10 Word2Vec vectors for each review as its input features fall slightly behind, registering an accuracy of 77.4%. These findings hint that, within the context of this particular classification challenge, the more straightforward Perceptron-TFIDF and SVM-Word2Vec models are more successful than the FNN models, which might see improvement with additional tuning.Compared to ternary, binary classification shows slightly better accuracy.</span>